In [56]:
import re
import pandas as pd
from json import dump
from collections import defaultdict
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests

In [74]:
postcode = pd.read_csv('../data/raw/Australian-Postcode-Data-master/au_postcodes.csv')

In [79]:
postcode['state_code'].value_counts()

NSW    5124
QLD    3456
VIC    3260
WA     1903
SA     1783
TAS     831
NT      364
ACT     154
Name: state_code, dtype: int64

In [105]:
code_vic = postcode[postcode['state_code'] == 'VIC']
code_vic = code_vic.drop_duplicates(subset=['longitude'])
codes = code_vic['postcode'].tolist()
len(codes)

1627

In [ ]:
# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 2) # update this to your liking

headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
# begin code
url_links = []
property_metadata = defaultdict(dict)

for code in codes: 
    # generate list of urls to visit
    for page in N_PAGES:
        url = BASE_URL + f"/rent/?postcode={code}&page={page}"
        #     print(url)
        bs_object = BeautifulSoup(requests.get(url, headers=headers).text, "html.parser")

        # find the unordered list (ul) elements which are the results, then
        # find all href (a) tags that are from the base_url website.
        index_links = bs_object \
            .find(
                "ul",
                {"data-testid": "results"}
            ) \
            .findAll(
                "a",
                href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
            )

        for link in index_links:
            # if its a property address, add it to the list
            if 'address' in link['class']:
                url_links.append(link['href'])

    # for each url, scrape some basic metadata
    for property_url in url_links[1:]:
        print(property_url)
        bs_object = BeautifulSoup(requests.get(property_url, headers=headers).text, "html.parser")

        # looks for the header class to get property name
        property_metadata[property_url]['name'] = bs_object \
            .find("h1", {"class": "css-164r41r"}) \
            .text

        # looks for the div containing a summary title for cost
        property_metadata[property_url]['cost_text'] = bs_object \
            .find("div", {"data-testid": "listing-details__summary-title"}) \
            .text

        # extract coordinates from the hyperlink provided
        # i'll let you figure out what this does :P
        property_metadata[property_url]['coordinates'] = [
            float(coord) for coord in re.findall(
                r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
                bs_object \
                    .find(
                        "a",
                        {"target": "_blank", 'rel': "noopener noreferer"}
                    ) \
                    .attrs['href']
            )[0].split(',')
        ]

        property_metadata[property_url]['rooms'] = [
            re.findall(r'\d\s[A-Za-z]+', feature.text) for feature in bs_object \
                .find("div", {"data-testid": "property-features"}) \
                .findAll("span", {"data-testid": "property-features-text-container"})
        ]

        property_metadata[property_url]['desc'] = re \
            .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
            .strip('</p>')


with open('../data/raw/domain.json', 'w') as f:
    dump(property_metadata, f)

In [51]:
# open route service demo
import openrouteservice as ors
client = ors.Client(key='5b3ce3597851110001cf6248da6ce7d997a5412896a30bc0c18e1e0a')

In [ ]:
coords = ((144.9614,-37.7963,),(144.843333,-37.673333))

routes = client.directions(coords, radiuses = [10000,10000])

print(routes)